In [1]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/251.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/251.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/251.3 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/251.3 kB 325.1 kB/s eta 0:00:01
   ------------------ ------------------- 122.9/251.3 kB 901.1 kB/s eta 0:00:01
   ---------------------------------------  245.8/251.3 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 251.3/251.3 kB 1.2 MB/s eta 0:00:00


In [4]:
!pip install tensorflow pandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.16.1-cp312-cp312-win_amd64.whl.metadata (3.5 kB)
  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached tensorflow_intel-2.16.1-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.3.2-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.me

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [5]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'pandas'

In [8]:
# Montez Google Drive pour accéder au fichier
from google.colab import drive
drive.mount('/content/drive')

# Chemin vers le fichier Excel
file_path = '/content/drive/My Drive/olympic_final.xlsx'

# Charger les données depuis le fichier Excel
df = pd.read_excel(file_path)

# Afficher les premières lignes pour vérifier que les données sont chargées correctement
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   discipline_title            event_title event_gender participant_type  \
0           Curling          Mixed Doubles        Mixed         GameTeam   
1           Curling          Mixed Doubles        Mixed         GameTeam   
2           Curling          Mixed Doubles        Mixed         GameTeam   
3  Freestyle Skiing  Men's Freeski Big Air          Men          Athlete   
4  Freestyle Skiing  Men's Freeski Big Air          Men          Athlete   

      athlete_full_name country_name  games_participations  \
0  Stefania CONSTANTINI        Italy                     1   
1         Almida DE VAL       Sweden                     1   
2        Oskar ERIKSSON       Sweden                     4   
3             Birk RUUD       Norway                     1   
4       Colby STEVENSON          USA                     1   

   athlete_year_birth  nb_gold  nb_silver

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Sélection des fonctionnalités pertinentes
features = ['country_name', 'nb_gold', 'nb_silver', 'nb_bronze']

# Séparation des fonctionnalités et de la cible
X = df[features]
y = df['Medals']

# Encodage des variables catégorielles
label_encoder = LabelEncoder()
X['country_name'] = label_encoder.fit_transform(X['country_name'])

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Affichage des premières lignes des ensembles d'entraînement et de test
print("Ensemble d'entraînement:")
print(X_train.head())
print("\nEnsemble de test:")
print(X_test.head())

Ensemble d'entraînement:
       country_name  nb_gold  nb_silver  nb_bronze
10999            86        3          5          1
4028              5        0          0          1
13984           103        0          1          0
10497            55        0          0          1
17293             5        0          3          0

Ensemble de test:
       country_name  nb_gold  nb_silver  nb_bronze
8739            138        0          0          1
13709            55        0          0          1
16354           119        0          0          1
13698            97        0          0          1
8377             88        0          1          0


<ipython-input-9-9dd654a00bfa>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['country_name'] = label_encoder.fit_transform(X['country_name'])


In [10]:
import tensorflow as tf

# Création du modèle
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compilation du modèle
model.compile(optimizer='adam',
              loss='mse', # Mean Squared Error
              metrics=['mae']) # Mean Absolute Error

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Évaluation du modèle
loss, mae = model.evaluate(X_test, y_test)
print("Mean Absolute Error sur l'ensemble de test:", mae)

# Prédiction des médailles pour de nouvelles données
predictions = model.predict(X_test[:5])
print("Prédictions des médailles:")
for i in range(len(predictions)):
    print("Pays:", X_test.iloc[i]['country_name'], "| Nombre de médailles prédit:", predictions[i][0], "| Nombre de médailles réel:", y_test.iloc[i])

Epoch 1/50
519/519 [==============================] - 2s 2ms/step - loss: 0.7492 - mae: 0.3592 - val_loss: 0.0023 - val_mae: 0.0207
Epoch 2/50
519/519 [==============================] - 1s 2ms/step - loss: 8.6189e-04 - mae: 0.0131 - val_loss: 3.7136e-04 - val_mae: 0.0099
Epoch 3/50
519/519 [==============================] - 1s 2ms/step - loss: 0.0049 - mae: 0.0312 - val_loss: 1.2489e-04 - val_mae: 0.0040
Epoch 4/50
519/519 [==============================] - 1s 2ms/step - loss: 0.0021 - mae: 0.0203 - val_loss: 0.0024 - val_mae: 0.0378
Epoch 5/50
519/519 [==============================] - 2s 3ms/step - loss: 0.0032 - mae: 0.0341 - val_loss: 8.0605e-05 - val_mae: 0.0065
Epoch 6/50
519/519 [==============================] - 2s 3ms/step - loss: 0.0026 - mae: 0.0260 - val_loss: 1.8772e-04 - val_mae: 0.0113
Epoch 7/50
519/519 [==============================] - 2s 3ms/step - loss: 0.0044 - mae: 0.0403 - val_loss: 0.0049 - val_mae: 0.0613
Epoch 8/50
519/519 [==============================] - 1s

In [45]:
# Affichage de l'accuracy
accuracy = 1 - loss
print("Accuracy sur l'ensemble de test:", accuracy)

Accuracy sur l'ensemble de test: 0.9999959415913509


In [12]:
# Inversez le processus d'encodage pour obtenir le nom du pays à partir du numéro encodé
inv_map = {i: label_encoder.classes_[i] for i in range(len(label_encoder.classes_))}

# Prédiction des médailles pour de nouvelles données avec le nom du pays
predictions = model.predict(X_test[:5])
print("Prédictions des médailles:")
for i in range(len(predictions)):
    country_name = inv_map[int(X_test.iloc[i]['country_name'])] # Obtenez le nom du pays à partir du numéro encodé
    print("Pays:", country_name, "| Nombre de médailles prédit:", predictions[i][0], "| Nombre de médailles réel:", y_test.iloc[i])

1/1 [==============================] - 0s 30ms/step
Prédictions des médailles:
Pays: Venezuela | Nombre de médailles prédit: 0.9998912 | Nombre de médailles réel: 1
Pays: Hungary | Nombre de médailles prédit: 0.999986 | Nombre de médailles réel: 1
Pays: Sweden | Nombre de médailles prédit: 0.9999119 | Nombre de médailles réel: 1
Pays: Poland | Nombre de médailles prédit: 0.9999378 | Nombre de médailles réel: 1
Pays: New Zealand | Nombre de médailles prédit: 1.0003816 | Nombre de médailles réel: 1


In [13]:
# Prédiction du nombre de médailles pour tous les pays
all_predictions = model.predict(X_test)

# Création d'un DataFrame pour stocker les prédictions et les pays correspondants
predictions_df = pd.DataFrame({'Country': X_test['country_name'], 'Predicted_Medals': all_predictions.flatten()})

# Agrégation des prédictions par pays pour obtenir le total des médailles prédites
predicted_totals = predictions_df.groupby('Country').sum()

# Trier les pays en fonction du total des médailles prédites
top_10_countries = predicted_totals.sort_values(by='Predicted_Medals', ascending=False).head(10)

print("Top 10 des pays avec le plus de médailles prédites:")
print(top_10_countries)

162/162 [==============================] - 1s 4ms/step
Top 10 des pays avec le plus de médailles prédites:
         Predicted_Medals
Country                  
130           1794.345947
46            1482.828735
103           1371.086792
64             547.946533
92             495.993683
43             421.980316
23             416.937347
55             413.881622
42             385.904236
119            375.028229


In [14]:
# Inversez le processus d'encodage pour obtenir le nom du pays à partir du numéro encodé
inv_map = {i: label_encoder.classes_[i] for i in range(len(label_encoder.classes_))}

# Création d'un DataFrame pour stocker les prédictions et les noms des pays correspondants
predictions_df = pd.DataFrame({'Country': X_test['country_name'].map(inv_map), 'Predicted_Medals': all_predictions.flatten()})

# Agrégation des prédictions par pays pour obtenir le total des médailles prédites
predicted_totals = predictions_df.groupby('Country').sum()

# Trier les pays en fonction du total des médailles prédites
top_10_countries = predicted_totals.sort_values(by='Predicted_Medals', ascending=False).head(10)

print("Top 10 des pays avec le plus de médailles prédites:")
print(top_10_countries)

Top 10 des pays avec le plus de médailles prédites:
         Predicted_Medals
Country                  
USA           1794.345947
Germany       1482.828735
Russia        1371.086792
Italy          547.946533
Norway         495.993683
France         421.980316
China          416.937347
Hungary        413.881622
Finland        385.904236
Sweden         375.028229
